# Análise do Risco Geo-Hidrológico e Acidentes

https://www.gov.br/cemaden/pt-br/assuntos/riscos-geo-hidrologicos

In [4]:
import geopandas as gpd
import fiona
from shapely.geometry import Point
import pandas as pd

In [5]:
# Read KML File
fiona.drvsupport.supported_drivers['KML'] = 'rw'
my_map = gpd.read_file('mapas/cemaden_25_12_2022.kml', driver='KML')
my_map

,Name,Description,geometry
0,Área de riscos (25.12.22),,"POLYGON Z ((-43.44546 -14.74019 0.00000, -43.2..."


In [6]:
# Read DataTran Base
data = pd.read_csv('raw_data/datatran_2022.csv')
data.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,405151,1/1/2022,sábado,1:35:00,PI,316.0,415.0,MARCOLANDIA,Ingestão de álcool pelo condutor,Colisão traseira,...,0,1,0,1,2,-7.432800,-40.682619,SPRF-PI,DEL04-PI,UOP03-DEL04-PI
1,405158,1/1/2022,sábado,2:40:00,PR,116.0,33.0,CAMPINA GRANDE DO SUL,Ingestão de álcool pelo condutor,Tombamento,...,0,0,0,1,1,-25.114403,-48.846755,SPRF-PR,DEL01-PR,UOP02-DEL01-PR
2,405172,1/1/2022,sábado,5:22:00,MS,163.0,393.0,NOVA ALVORADA DO SUL,Condutor deixou de manter distância do veículo...,Colisão traseira,...,0,1,0,1,2,-21.228445,-54.456296,SPRF-MS,DEL02-MS,UOP01-DEL02-MS
3,405203,1/1/2022,sábado,7:00:00,RJ,101.0,457.0,ANGRA DOS REIS,Reação tardia ou ineficiente do condutor,Colisão frontal,...,1,1,0,1,2,-23.031498,-44.177153,SPRF-RJ,DEL03-RJ,UOP02-DEL03-RJ
4,405207,1/1/2022,sábado,9:00:00,MG,40.0,508.3,RIBEIRAO DAS NEVES,Acumulo de água sobre o pavimento,Saída de leito carroçável,...,0,3,0,0,1,-19.760612,-44.134754,SPRF-MG,DEL02-MG,UOP01-DEL02-MG


In [13]:
# Capturing geo information from accidents in 25/12/2022
new_df = data.loc[data['data_inversa'] == '12/25/2022'][['uf','longitude', 'latitude', "causa_acidente"]]
new_df.head()

,uf,longitude,latitude,causa_acidente
16410,SE,-36.982008,-10.619092,Transitar na contramão
16412,CE,-40.135711,-3.793610,Curva acentuada
16413,SC,-52.861614,-26.919464,Velocidade Incompatível
16415,TO,-49.159510,-12.262294,Condutor Dormindo
16416,GO,-48.701858,-15.200917,Demais falhas mecânicas ou elétricas


In [14]:
# Registring as Geometric Points
geometic_points = []
for xy in zip(new_df['longitude'], new_df['latitude']):
    geometic_points.append(Point(xy))
    
geo_locations = gpd.GeoDataFrame(new_df,
                                 crs = {'init': 'epsg:4326'},
                                 geometry = geometic_points)

geo_locations

/Users/ciziks/Documentos/USP/Semestre 6/PCD_2/traffic-accident-analysis/.venv/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,uf,longitude,latitude,causa_acidente,geometry
16410,SE,-36.982008,-10.619092,Transitar na contramão,POINT (-36.98201 -10.61909)
16412,CE,-40.135711,-3.793610,Curva acentuada,POINT (-40.13571 -3.79361)
16413,SC,-52.861614,-26.919464,Velocidade Incompatível,POINT (-52.86161 -26.91946)
16415,TO,-49.159510,-12.262294,Condutor Dormindo,POINT (-49.15951 -12.26229)
16416,GO,-48.701858,-15.200917,Demais falhas mecânicas ou elétricas,POINT (-48.70186 -15.20092)
...,...,...,...,...,...
64322,MT,-55.888555,-13.004629,Ultrapassagem Indevida,POINT (-55.88855 -13.00463)
64452,MG,-43.252280,-16.415009,Condutor Dormindo,POINT (-43.25228 -16.41501)
64454,PR,-52.139397,-23.289445,Animais na Pista,POINT (-52.13940 -23.28945)
64493,PR,-51.931583,-23.412692,Reação tardia ou ineficiente do condutor,POINT (-51.93158 -23.41269)


In [15]:
geo_locations['in_my_map'] = ''

In [16]:
for i in geo_locations.index:
    point = geo_locations.geometry.loc[i]
    check = my_map.contains(point).values
    geo_locations['in_my_map'].loc[i] = check
geo_locations.head(5)

/var/folders/p9/dn9xbhl57ns3gzlc5zxhvfg40000gn/T/ipykernel_4913/3285326073.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_locations['in_my_map'].loc[i] = check


,uf,longitude,latitude,causa_acidente,geometry,in_my_map
16410,SE,-36.982008,-10.619092,Transitar na contramão,POINT (-36.98201 -10.61909),False
16412,CE,-40.135711,-3.793610,Curva acentuada,POINT (-40.13571 -3.79361),False
16413,SC,-52.861614,-26.919464,Velocidade Incompatível,POINT (-52.86161 -26.91946),False
16415,TO,-49.159510,-12.262294,Condutor Dormindo,POINT (-49.15951 -12.26229),False
16416,GO,-48.701858,-15.200917,Demais falhas mecânicas ou elétricas,POINT (-48.70186 -15.20092),False


In [19]:
geo_locations[geo_locations.in_my_map == True]

,uf,longitude,latitude,causa_acidente,geometry,in_my_map
63192,BA,-39.76892,-17.635162,Ingestão de álcool pelo condutor,POINT (-39.76892 -17.63516),True


In [17]:
print(sum(geo_locations.in_my_map == True))

1


In [18]:
print(sum(geo_locations.in_my_map == False))

203
